# Projeto TMDB – Coleta, Tratamento e Análise (Notebook PRO)

Este notebook implementa um pipeline completo usando a API pública do TMDB:

1. Conexão com a API (filmes, séries e trending)
2. Coleta paginada da base principal (`df_bruto`)
3. Coleta de **detalhes por ID** (orçamento, receita, países, duração etc.)
4. Tratamento, limpeza e **engenharia de atributos** (`df_modelagem`)
5. Enriquecimento com:
   - País de produção
   - Gêneros (nomes)
   - Duração aproximada
   - Lucro (receita – orçamento)
6. Criação de visualizações avançadas:
   - Distribuição por ano
   - Distribuição de notas e popularidade
   - Dispersão popularidade x nota
   - Distribuição de gêneros (nomes)
   - Países de produção (barras + mapa mundial)
   - Mapa de correlação entre variáveis


In [1]:
# Imports e configurações

import os
import time
import ast
from typing import Dict, List, Any, Optional

import requests
import numpy as np
import pandas as pd
import plotly.express as px

# ============================
# CONFIGURAÇÕES GERAIS
# ============================

# 🔑 Substitua pela sua chave v3 do TMDB (NÃO compartilhe essa chave)
TMDB_API_KEY = "19e1cd0669df57e8ba8b78cd5458f263"

TMDB_BASE_URL = "https://api.themoviedb.org/3"
LANGUAGE = "pt-BR"

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

print("📁 BASE_DIR:", BASE_DIR)
print("📁 DATA_DIR:", DATA_DIR)

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 180)


📁 BASE_DIR: c:\Projetos Ciência de Dados\tmdb
📁 DATA_DIR: c:\Projetos Ciência de Dados\tmdb\data


In [2]:
# ============================
# FUNÇÕES AUXILIARES TMDB
# ============================

def tmdb_get(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Chama a API TMDB e retorna o JSON, com tratamento básico de erro."""
    if params is None:
        params = {}
    params["api_key"] = TMDB_API_KEY
    params["language"] = LANGUAGE

    url = f"{TMDB_BASE_URL}{endpoint}"
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r.json()


def collect_paginated(endpoint: str, pages: int = 5, extra_params: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
    """Coleta várias páginas de um endpoint paginado do TMDB."""
    all_results: List[Dict[str, Any]] = []
    params = extra_params.copy() if extra_params else {}

    for page in range(1, pages + 1):
        params["page"] = page
        data = tmdb_get(endpoint, params)
        results = data.get("results", [])
        all_results.extend(results)

        total_pages = data.get("total_pages", 1)
        print(f"[{endpoint}] página {page}/{total_pages} – +{len(results)} itens")
        if page >= total_pages:
            break

        time.sleep(0.3)  # evita flood de requisições

    return all_results


def get_genre_mapping() -> Dict[int, str]:
    """Retorna um dicionário {id_genero: nome_genero} combinando filmes e séries."""
    movie_genres = tmdb_get("/genre/movie/list").get("genres", [])
    tv_genres = tmdb_get("/genre/tv/list").get("genres", [])
    mapping: Dict[int, str] = {}
    for g in movie_genres + tv_genres:
        mapping[g["id"]] = g["name"]
    return mapping


def get_details_for_items(df_ids: pd.DataFrame, max_items: int = 150) -> pd.DataFrame:
    """
    Coleta detalhes adicionais para um subconjunto de títulos (filmes e séries).

    max_items: limite para não ficar rodando por muitos minutos.
    """
    detalhes: List[Dict[str, Any]] = []
    subset = df_ids.drop_duplicates(subset=["id"]).head(max_items)

    print(f"Coletando detalhes para até {len(subset)} títulos...")

    for i, (_, row) in enumerate(subset.iterrows(), start=1):
        mid = row["id"]
        tipo = str(row["tipo"])

        if i % 20 == 0 or i == 1:
            print(f"  → Detalhes {i}/{len(subset)} (id={mid}, tipo={tipo})")

        try:
            if "Filme" in tipo:
                ep = f"/movie/{mid}"
            elif "Série" in tipo:
                ep = f"/tv/{mid}"
            else:
                continue

            d = tmdb_get(ep)
            d["id"] = mid
            detalhes.append(d)
            time.sleep(0.3)
        except Exception as e:
            print(f"    ⚠️ Erro ao buscar detalhes para id={mid}: {e}")

    if not detalhes:
        return pd.DataFrame()

    return pd.json_normalize(detalhes)


In [3]:
# ============================
# COLETA PRINCIPAL
# ============================

# Filmes populares
filmes = collect_paginated("/movie/popular", pages=5)
for f in filmes:
    f["tipo"] = "Filme (Popular)"

# Séries populares
series = collect_paginated("/tv/popular", pages=5)
for s in series:
    s["tipo"] = "Série (Popular)"

# Trending (dia)
trending = collect_paginated("/trending/all/day", pages=5)
for t in trending:
    mt = t.get("media_type", "")
    if mt == "movie":
        t["tipo"] = "Filme (Trending)"
    elif mt == "tv":
        t["tipo"] = "Série (Trending)"
    else:
        t["tipo"] = "Outro (Trending)"

todos = filmes + series + trending
df_bruto = pd.json_normalize(todos)

print("Shape df_bruto:", df_bruto.shape)
df_bruto.head()


[/movie/popular] página 1/53871 – +20 itens
[/movie/popular] página 2/53878 – +20 itens
[/movie/popular] página 3/53871 – +20 itens
[/movie/popular] página 4/53877 – +20 itens
[/movie/popular] página 5/53877 – +20 itens
[/tv/popular] página 1/10473 – +20 itens
[/tv/popular] página 2/10473 – +20 itens
[/tv/popular] página 3/10474 – +20 itens
[/tv/popular] página 4/10473 – +20 itens
[/tv/popular] página 5/10474 – +20 itens
[/trending/all/day] página 1/500 – +20 itens
[/trending/all/day] página 2/500 – +20 itens
[/trending/all/day] página 3/500 – +20 itens
[/trending/all/day] página 4/500 – +20 itens
[/trending/all/day] página 5/500 – +20 itens
Shape df_bruto: (300, 23)


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,tipo,origin_country,original_name,first_air_date,name,media_type,gender,known_for_department,profile_path
0,False,/4BtL2vvEufDXDP4u6xQjjQ1Y2aT.jpg,"[28, 80, 53]",1419406,zh,捕风追影,A polícia de Macau remove o policial que é esp...,501.8155,/uLmjQTgqXtuXRPOWfHB4vOmbgzt.jpg,2025-08-16,A Sombra Do Perigo,False,6.400,99.0,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,/tQMlHruS4pU2PKLf9CgWwkFR399.jpg,"[28, 12, 878]",1033462,zh,749局,,499.7803,/xW640PVBXLlzhrkQnAcvWNsehIO.jpg,2024-10-01,749局,False,5.387,31.0,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,/93mQVskuy8kZOXPoLve8P2E5zdD.jpg,"[28, 35]",1363123,en,The Family Plan 2,Agora que os dias de assassino de Dan ficaram ...,416.9137,/aLgvLNWETZ2wtPzU3E7lavEpCJw.jpg,2025-11-11,Plano em Família 2,False,6.777,157.0,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,/zEsHEpCGZwGg3M2b0oSZuaPLwBh.jpg,"[878, 28]",1309012,en,Altered,"Num presente alternativo, humanos geneticament...",330.9811,/6QlAcGRaUrgHcZ4WTBh5lsPnzKx.jpg,2025-09-18,Altered,False,6.800,29.0,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,/hpXBJxLD2SEf8l2CspmSeiHrBKX.jpg,"[18, 27, 14]",1062722,en,Frankenstein,"Dr. Victor Frankenstein, um cientista brilhant...",275.5065,/cXsMxClCcAF1oMwoXZvbKwWoNeS.jpg,2025-10-17,Frankenstein,False,7.772,1855.0,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Salvar df_bruto

df_bruto_path = os.path.join(DATA_DIR, "df_bruto.csv")
df_bruto.to_csv(df_bruto_path, index=False, encoding="utf-8")
print("✔ df_bruto salvo em:", df_bruto_path)


✔ df_bruto salvo em: c:\Projetos Ciência de Dados\tmdb\data\df_bruto.csv


In [5]:
# ============================
# GÊNEROS E DETALHES
# ============================

genre_map = get_genre_mapping()
print("Alguns gêneros:", list(genre_map.items())[:10])

# IDs para detalhes
df_ids = df_bruto[["id", "tipo"]].drop_duplicates()
df_det = get_details_for_items(df_ids, max_items=80)

print("Shape df_det:", df_det.shape)
cols_preview = [c for c in [
    "id",
    "runtime",
    "episode_run_time",
    "number_of_seasons",
    "number_of_episodes",
    "budget",
    "revenue",
    "production_countries",
    "origin_country"
] if c in df_det.columns]

df_det[cols_preview].head()


Alguns gêneros: [(28, 'Ação'), (12, 'Aventura'), (16, 'Animação'), (35, 'Comédia'), (80, 'Crime'), (99, 'Documentário'), (18, 'Drama'), (10751, 'Família'), (14, 'Fantasia'), (36, 'História')]
Coletando detalhes para até 80 títulos...
  → Detalhes 1/80 (id=1419406, tipo=Filme (Popular))
  → Detalhes 20/80 (id=425274, tipo=Filme (Popular))
  → Detalhes 40/80 (id=1025527, tipo=Filme (Popular))
  → Detalhes 60/80 (id=297802, tipo=Filme (Popular))
  → Detalhes 80/80 (id=338969, tipo=Filme (Popular))
Shape df_det: (80, 30)


,id,runtime,budget,revenue,production_countries,origin_country
0,1419406,142,0,702538,"[{'iso_3166_1': 'CN', 'name': 'China'}]","[CN, HK]"
1,1033462,123,0,51706251,"[{'iso_3166_1': 'CN', 'name': 'China'}]",[CN]
2,1363123,106,0,0,"[{'iso_3166_1': 'US', 'name': 'United States o...",[US]
3,1309012,85,15000000,0,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",[CA]
4,1062722,149,120000000,480678,"[{'iso_3166_1': 'US', 'name': 'United States o...",[US]


In [6]:
# ============================
# TRATAMENTO E ENGENHARIA
# ============================

def tratar_e_enriquecer(df_raw: pd.DataFrame, df_det: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # 1) Título único (filmes e séries)
    df["titulo_unico"] = (
        df.get("title", "").fillna("").astype(str).str.strip()
        .mask(lambda s: s == "", df.get("name", "").fillna("").astype(str).str.strip())
    )

    # 2) Ano de lançamento
    release = pd.to_datetime(df.get("release_date"), errors="coerce")
    first_air = pd.to_datetime(df.get("first_air_date"), errors="coerce")
    df["ano_lancamento"] = release.dt.year.fillna(first_air.dt.year)
    df = df.dropna(subset=["ano_lancamento"])
    df["ano_lancamento"] = df["ano_lancamento"].astype(int)

    # 3) Sinopse e tamanho (nº de palavras)
    df["overview"] = df.get("overview", "").fillna("").astype(str)
    df["tamanho_sinopse"] = df["overview"].apply(lambda x: len(x.split()))

    # 4) Popularidade e logs
    df["popularity"] = df.get("popularity", 0).fillna(0).astype(float)
    df["popularidade_log"] = np.log1p(df["popularity"])

    # 5) Notas e votos
    df["vote_average"] = df.get("vote_average", 0).fillna(0).astype(float)
    df["vote_count"] = df.get("vote_count", 0).fillna(0).astype(int)
    max_votos = max(df["vote_count"].max(), 1)
    df["nota_ponderada"] = df["vote_average"] * (df["vote_count"] / max_votos)
    df["vote_count_log"] = np.log1p(df["vote_count"])

    # 6) Categoria de popularidade (quartis)
    try:
        df["popularidade_categoria"] = pd.qcut(
            df["popularity"],
            4,
            labels=["Baixa", "Média", "Alta", "Muito Alta"]
        )
    except ValueError:
        df["popularidade_categoria"] = "Indefinida"

    # 7) genre_ids → lista + nomes
    def parse_genres(x):
        if isinstance(x, list):
            return x
        if isinstance(x, str):
            try:
                val = ast.literal_eval(x)
                return val if isinstance(val, list) else None
            except Exception:
                return None
        return None

    if "genre_ids" in df.columns:
        df["genre_ids"] = df["genre_ids"].apply(parse_genres)
    else:
        df["genre_ids"] = None

    def decode_genre_names(ids):
        if not isinstance(ids, list):
            return []
        return [genre_map.get(i, str(i)) for i in ids]

    df["generos_nomes"] = df["genre_ids"].apply(decode_genre_names)

    # 8) Merge com detalhes (países, orçamento, receita, duração)
    if not df_det.empty and "id" in df_det.columns:
        cols_keep = [c for c in [
            "id",
            "runtime",
            "episode_run_time",
            "number_of_seasons",
            "number_of_episodes",
            "budget",
            "revenue",
            "production_countries",
            "origin_country"
        ] if c in df_det.columns]

        df_det_slim = df_det[cols_keep].drop_duplicates(subset=["id"])
        df = df.merge(df_det_slim, on="id", how="left")

        # Duração aproximada em minutos
        def obter_duracao(row):
            if "runtime" in row and pd.notna(row["runtime"]):
                return row["runtime"]
            ert = row.get("episode_run_time")
            if isinstance(ert, list) and ert:
                return ert[0]
            return np.nan

        df["duracao_min"] = df.apply(obter_duracao, axis=1)

        # País código e nome
        def obter_pais_codigo(row):
            pc = row.get("production_countries")
            if isinstance(pc, list) and pc:
                c0 = pc[0]
                if isinstance(c0, dict):
                    return c0.get("iso_3166_1")
            oc = row.get("origin_country")
            if isinstance(oc, list) and oc:
                return oc[0]
            return np.nan

        def obter_pais_nome(row):
            pc = row.get("production_countries")
            if isinstance(pc, list) and pc:
                c0 = pc[0]
                if isinstance(c0, dict):
                    return c0.get("name")
            return np.nan

        df["pais_codigo"] = df.apply(obter_pais_codigo, axis=1)
        df["pais_nome"] = df.apply(obter_pais_nome, axis=1)

        # Orçamento, receita, lucro
        df["budget"] = df.get("budget", 0).fillna(0).astype(float)
        df["revenue"] = df.get("revenue", 0).fillna(0).astype(float)
        df["lucro"] = df["revenue"] - df["budget"]
        df["lucro_log"] = np.log1p(df["lucro"].clip(lower=0))
    else:
        df["duracao_min"] = np.nan
        df["pais_codigo"] = np.nan
        df["pais_nome"] = np.nan
        df["budget"] = 0.0
        df["revenue"] = 0.0
        df["lucro"] = 0.0
        df["lucro_log"] = 0.0

    # 9) Remover duplicados
    if "id" in df.columns:
        df = df.drop_duplicates(subset=["id", "tipo"])

    return df


df_modelagem = tratar_e_enriquecer(df_bruto, df_det)
print("Shape df_modelagem:", df_modelagem.shape)
df_modelagem.head()


Shape df_modelagem: (286, 41)


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,tipo,origin_country_x,original_name,first_air_date,name,media_type,gender,known_for_department,profile_path,titulo_unico,ano_lancamento,tamanho_sinopse,popularidade_log,nota_ponderada,vote_count_log,popularidade_categoria,generos_nomes,runtime,budget,revenue,production_countries,origin_country_y,duracao_min,pais_codigo,pais_nome,lucro,lucro_log
0,False,/4BtL2vvEufDXDP4u6xQjjQ1Y2aT.jpg,"[28, 80, 53]",1419406,zh,捕风追影,A polícia de Macau remove o policial que é esp...,501.8155,/uLmjQTgqXtuXRPOWfHB4vOmbgzt.jpg,2025-08-16,A Sombra Do Perigo,False,6.400,99,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A Sombra Do Perigo,2025,24,6.220223,0.016553,4.605170,Muito Alta,"[Ação, Crime, Thriller]",142.0,0.0,702538.0,"[{'iso_3166_1': 'CN', 'name': 'China'}]","[CN, HK]",142.0,CN,China,702538.0,13.462456
1,False,/tQMlHruS4pU2PKLf9CgWwkFR399.jpg,"[28, 12, 878]",1033462,zh,749局,,499.7803,/xW640PVBXLlzhrkQnAcvWNsehIO.jpg,2024-10-01,749局,False,5.387,31,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,749局,2024,0,6.216167,0.004363,3.465736,Muito Alta,"[Ação, Aventura, Ficção científica]",123.0,0.0,51706251.0,"[{'iso_3166_1': 'CN', 'name': 'China'}]",[CN],123.0,CN,China,51706251.0,17.761089
2,False,/93mQVskuy8kZOXPoLve8P2E5zdD.jpg,"[28, 35]",1363123,en,The Family Plan 2,Agora que os dias de assassino de Dan ficaram ...,416.9137,/aLgvLNWETZ2wtPzU3E7lavEpCJw.jpg,2025-11-11,Plano em Família 2,False,6.777,157,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plano em Família 2,2025,53,6.035275,0.027796,5.062595,Muito Alta,"[Ação, Comédia]",106.0,0.0,0.0,"[{'iso_3166_1': 'US', 'name': 'United States o...",[US],106.0,US,United States of America,0.0,0.000000
3,False,/zEsHEpCGZwGg3M2b0oSZuaPLwBh.jpg,"[878, 28]",1309012,en,Altered,"Num presente alternativo, humanos geneticament...",330.9811,/6QlAcGRaUrgHcZ4WTBh5lsPnzKx.jpg,2025-09-18,Altered,False,6.800,29,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Altered,2025,32,5.805078,0.005152,3.401197,Muito Alta,"[Ficção científica, Ação]",85.0,15000000.0,0.0,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",[CA],85.0,CA,Canada,-15000000.0,0.000000
4,False,/hpXBJxLD2SEf8l2CspmSeiHrBKX.jpg,"[18, 27, 14]",1062722,en,Frankenstein,"Dr. Victor Frankenstein, um cientista brilhant...",275.5065,/cXsMxClCcAF1oMwoXZvbKwWoNeS.jpg,2025-10-17,Frankenstein,False,7.772,1855,Filme (Popular),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frankenstein,2025,30,5.622234,0.376641,7.526179,Muito Alta,"[Drama, Terror, Fantasia]",149.0,120000000.0,480678.0,"[{'iso_3166_1': 'US', 'name': 'United States o...",[US],149.0,US,United States of America,-119519322.0,0.000000


In [7]:
# Salvar df_modelagem

df_modelagem_path = os.path.join(DATA_DIR, "df_modelagem.csv")
df_modelagem.to_csv(df_modelagem_path, index=False, encoding="utf-8")
print("✔ df_modelagem salvo em:", df_modelagem_path)


✔ df_modelagem salvo em: c:\Projetos Ciência de Dados\tmdb\data\df_modelagem.csv


In [8]:
# Estatísticas descritivas

print("Total de títulos:", len(df_modelagem))

print("\nPor tipo:")
print(df_modelagem["tipo"].value_counts())

print("\nResumo de notas, votos, popularidade e lucro:")
print(
    df_modelagem[[
        "vote_average", "vote_count", "nota_ponderada",
        "popularity", "popularidade_log", "lucro"
    ]].describe()
)


Total de títulos: 286

Por tipo:
tipo
Filme (Popular)     97
Série (Popular)     90
Filme (Trending)    59
Série (Trending)    40
Name: count, dtype: int64

Resumo de notas, votos, popularidade e lucro:
       vote_average    vote_count  nota_ponderada  popularity  popularidade_log         lucro
count    286.000000    286.000000      286.000000  286.000000        286.000000  2.860000e+02
mean       7.103455   2751.933566        0.581147   79.906710          4.042791  4.325571e+07
std        1.636532   5883.809442        1.275454   80.944693          0.865199  1.694196e+08
min        0.000000      0.000000        0.000000    1.325500          0.843935 -1.800000e+08
25%        6.700000     40.000000        0.007024   37.034775          3.638484  0.000000e+00
50%        7.471500    258.500000        0.050355   56.333750          4.048889  0.000000e+00
75%        8.097500   2539.750000        0.519753   94.141700          4.555367  0.000000e+00
max       10.000000  38278.000000        8.49

In [9]:
# Distribuição por ano

df_year = (
    df_modelagem.groupby("ano_lancamento")
    .size()
    .reset_index(name="quantidade")
    .sort_values("ano_lancamento")
)

fig_ano = px.bar(
    df_year,
    x="ano_lancamento",
    y="quantidade",
    title="Quantidade de títulos por ano de lançamento"
)
fig_ano.update_layout(template="plotly_white")
fig_ano.show()


In [10]:
# Distribuição por notas e popularidade

fig_nota = px.histogram(
    df_modelagem,
    x="vote_average",
    nbins=20,
    title="Distribuição de notas médias"
)
fig_nota.update_layout(template="plotly_white")
fig_nota.show()

fig_pop = px.histogram(
    df_modelagem,
    x="popularity",
    nbins=20,
    title="Distribuição de popularidade"
)
fig_pop.update_layout(template="plotly_white")
fig_pop.show()


In [11]:
# Popularidade (log)  X nota 

fig_scatter = px.scatter(
    df_modelagem,
    x="popularidade_log",
    y="vote_average",
    color="tipo",
    size="popularity",
    hover_name="titulo_unico",
    opacity=0.7,
    title="Popularidade (log) × Nota média por tipo"
)
fig_scatter.update_layout(template="plotly_white")
fig_scatter.show()


In [12]:
# Distribuição de gêneros (nomes)

print("Colunas em df_modelagem:", df_modelagem.columns.tolist())

if "generos_nomes" not in df_modelagem.columns:
    print("❌ Coluna 'generos_nomes' não encontrada.")
else:
    exploded_gen = df_modelagem["generos_nomes"].explode().dropna()

    if exploded_gen.empty:
        print("⚠️ Nenhum gênero disponível para análise.")
    else:
        freq_gen = exploded_gen.value_counts().reset_index()
        freq_gen.columns = ["generos_nomes", "quantidade"]

        fig_gen = px.bar(
            freq_gen.sort_values("quantidade", ascending=True),
            x="quantidade",
            y="generos_nomes",
            orientation="h",
            title="🎭 Distribuição de gêneros (nomes)",
            text="quantidade",
            color="quantidade",
            color_continuous_scale="Blues"
        )
        fig_gen.update_layout(
            template="plotly_white",
            xaxis_title="Quantidade",
            yaxis_title="Gênero",
            coloraxis_showscale=False
        )
        fig_gen.show()


Colunas em df_modelagem: ['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count', 'tipo', 'origin_country_x', 'original_name', 'first_air_date', 'name', 'media_type', 'gender', 'known_for_department', 'profile_path', 'titulo_unico', 'ano_lancamento', 'tamanho_sinopse', 'popularidade_log', 'nota_ponderada', 'vote_count_log', 'popularidade_categoria', 'generos_nomes', 'runtime', 'budget', 'revenue', 'production_countries', 'origin_country_y', 'duracao_min', 'pais_codigo', 'pais_nome', 'lucro', 'lucro_log']


In [13]:
# Países de produção (barras) + mapa mundial

# Agregação por país
df_country = (
    df_modelagem.dropna(subset=["pais_nome"])
    .groupby("pais_nome", as_index=False)
    .agg(
        quantidade=("id", "count"),
        nota_media=("vote_average", "mean"),
        popularidade_media=("popularity", "mean"),
        lucro_medio=("lucro", "mean")
    )
    .sort_values("quantidade", ascending=False)
)

df_country.head()


,pais_nome,quantidade,nota_media,popularidade_media,lucro_medio
16,United States of America,62,6.845742,104.795526,1.707799e+08
11,Japan,7,6.337714,119.614957,2.527429e+08
4,China,6,6.256833,326.690883,2.565786e+07
2,Canada,5,5.959000,152.986960,-6.400000e+06
14,Thailand,3,5.116667,83.610067,0.000000e+00


In [14]:
# Barras – Top 20 países por quantidade
top_pais = df_country.head(20)

fig_pais = px.bar(
    top_pais.sort_values("quantidade"),
    x="quantidade",
    y="pais_nome",
    orientation="h",
    title="🌍 Top países de produção (por quantidade de títulos)"
)
fig_pais.update_layout(template="plotly_white")
fig_pais.show()


In [15]:
# Barras – Top 20 países por quantidade
top_pais = df_country.head(20)

fig_pais = px.bar(
    top_pais.sort_values("quantidade"),
    x="quantidade",
    y="pais_nome",
    orientation="h",
    title="🌍 Top países de produção (por quantidade de títulos)"
)
fig_pais.update_layout(template="plotly_white")
fig_pais.show()


In [16]:
# Correlação entre variáveis numéricas

cols_corr = [
    "vote_average",
    "vote_count",
    "nota_ponderada",
    "popularity",
    "popularidade_log",
    "tamanho_sinopse",
    "duracao_min",
    "budget",
    "revenue",
    "lucro"
]
cols_corr = [c for c in cols_corr if c in df_modelagem.columns]

corr = df_modelagem[cols_corr].corr()

fig_corr = px.imshow(
    corr,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Mapa de correlação – principais variáveis numéricas"
)
fig_corr.update_layout(template="plotly_white")
fig_corr.show()
